In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
from CHAID import Tree
import ast
from tqdm import tqdm
tqdm.pandas()
import re

In [3]:
binning_dict = dict()

In [4]:
import CHAID

In [8]:
dataset = pd.read_excel("C:/Users/88001381/Desktop/test.xlsx")
print(dataset.shape)
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

(57881, 87)


In [123]:
independent_variable_columns = ['BRANCH_NEW']
dep_variable = ["target_variable"]

tree = Tree.from_pandas_df(dataset, dict(zip(independent_variable_columns, ['nominal'] * 1)), 
                           dep_variable)
a = tree.tree_store[0]
a = ast.literal_eval(a.split.groupings)

def ApplicationUsage(x,a):
    y = ""
    for i,value in enumerate(a):
        if(x in value):
            y = i
    return(y)
    
dataset["BRANCH_NEW_B"] = dataset["BRANCH_NEW"].fillna('<missing>').apply(lambda x: ApplicationUsage(x,a))


In [124]:
temp = dict()
for i,value in enumerate(a):
    temp[str(i)] = value
    
x = "BRANCH_NEW_B"
for i in temp.keys():
    if(len(dataset[dataset[x].isin([int(i)])]["target_variable"].value_counts()) == 2):
        a1 = dataset[dataset[x].isin([int(i)])]["target_variable"].value_counts()[1]
        a2 = dataset[dataset[x].isin([int(i)])]["target_variable"].value_counts()[0]
        temp[i].append(["YES: " + str(a1) + " || NO: " + str(a2) + " || Bad%: " + str(round(a1/(a1+a2),4)*100)[0:5] + "%"])
    else:
        if(dataset[dataset[x].isin([int(i)])]["target_variable"].value_counts().index[0] == 0):
            a2 = dataset[dataset[x].isin([int(i)])]["target_variable"].value_counts()[0]
            temp[i].append(["YES: " + str(0) + " || NO: " + str(a2) + " || Bad%: " + str(round(a1/(a1+a2),4)*100)[0:5] + "%"])
        else:
            a1 = dataset[dataset[x].isin([int(i)])]["target_variable"].value_counts()[1]
            temp[i].append(["YES: " + str(a1) + " || NO: " + str(0) + " || Bad%: " + str(round(a1/(a1+a2),4)*100)[0:5] + "%"])
    
    
binning_dict["BRANCH_NEW"] = temp 


In [125]:
binning1 = pd.DataFrame(columns = ["Group","Binning","YES","NO"])
for j in binning_dict.keys():
    x = []
    for i in binning_dict[j].keys():
        try: 
            temp1 = binning_dict[j][i]
            temp = [i,''.join(str(temp1[:len(temp1)-1])),re.findall(r'\d+',temp1[-1][0])[0],re.findall(r'\d+',temp1[-1][0])[1]]
            x.append(temp)
        except:
            temp = [str(i),','.join(temp1[:len(temp1)-1]),"0","1"]
            x.append(temp)
    temp2 = pd.DataFrame(x,columns = ["Group","Binning","YES","NO"])
    temp2["Variable"] = [j for i in range(len(binning_dict[j].keys()))]
    temp2["Bad%"] = temp2["YES"].astype("int")/(temp2["NO"].astype("int")+temp2["YES"].astype("int"))
    binning1 =pd.concat([binning1,temp2],axis =0)
binning1.insert(0, "Variables", binning1["Variable"])
binning1 = binning1.drop(["Variable"],axis =1)


In [126]:
binning1

,Variables,Group,Binning,YES,NO,Bad%
0,NORM,0,"['ACE 7', 'PC01', 'PC04', 'RTC2', 'SRTC', 'WIN...",5,351,0.014045
1,NORM,1,"['CAR1', 'SALG', 'RTC1', 'RTL', 'SEP', 'SFLT',...",1069,12050,0.081485
2,NORM,2,"['CPV', 'SENP', 'FTU1']",3071,26599,0.103505
3,NORM,3,"['FTU2', 'SRT2', 'SRT1', 'STR']",508,2949,0.146948
4,NORM,4,"['MFLT', 'SSTR']",49,144,0.253886
...,...,...,...,...,...,...
8,BRANCH_NEW,8,"['BAFNA', 'BANDERDEWA', 'BANGALORE - KENGERI',...",0,2733,0.000000
9,BRANCH_NEW,9,"['BANGALORE - ELECTRONIC CITY', 'RANCHI', 'SIL...",542,4758,0.102264
10,BRANCH_NEW,10,"['BARMER', 'BHAVNAGAR', 'BOKARO', 'SHIMOGA', '...",643,4037,0.137393
11,BRANCH_NEW,11,"['BARUIPUR', 'BERAHAMPUR', 'NALGONDA', 'NANDED...",533,2935,0.153691


In [127]:
binning1.to_excel('testbinning.xlsx', index=False)

In [132]:
dataset.to_csv('test_with_binning.csv', index=False)

PermissionError: [Errno 13] Permission denied: 'test_with_binning.csv'

In [130]:
dataset.columns.values.tolist()

['Unnamed: 0',
 'CONNO',
 'Month',
 'FIN_AMOUNT',
 'PRODUCT',
 'NORM',
 'IRR_CUSTOMER',
 'APPLICATION_USAGE',
 'ASSET_CATEGORY2',
 'STATE',
 'BRANCH_NEW',
 'REGION_NEW',
 'ASSET_COST',
 'LTV',
 'CUSTOMER_SEGMENT',
 'COMPANY_INDIVIDUAL',
 'DSA_DLR',
 'SOURCING',
 'PDC_FLAG',
 'SCHEME_FINPROD',
 'COMPANY_CODE',
 'COLLECTION_TILL_DATE',
 'SOHP',
 'TENURE',
 'ODAMT',
 'COLLECTION_MONTH',
 'FIRST_EMI',
 'EMI',
 'TRACK_RECORD',
 'ODBKT',
 'STATUS',
 'INCEPTION_MONTH',
 'MOB',
 'SOHPOD',
 'SOHPOD_PREASSIGN',
 'SOHPOD_MOB1PLUS',
 'SOHPOD_MOB2PLUS',
 'SOHPOD_MOB3PLUS',
 'SOHPOD_MOB4PLUS',
 'SOHPOD_MOB5PLUS',
 'SOHPOD_MOB6PLUS',
 'Restructuring_Flag',
 'SEGMENT',
 'PAID_PER',
 'TENURE_LEFT',
 'ME',
 'CE',
 'max_odbkt',
 'Ever_30',
 'Ever_60',
 'Ever_90',
 'DELSTRING_PAST24M',
 'DELSTRING_PAST24M_WOCOVID',
 'max_dpd',
 'target_variable',
 'XBktString_L24M',
 'EverInBkt_1_3M',
 'EverInBkt_2_3M',
 'EverInBkt_3_3M',
 'EverInBkt_4Plus_3M',
 'EverInBkt_1_6M',
 'EverInBkt_2_6M',
 'EverInBkt_3_6M',
 'Ev